In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import NegotiationTools as negtools
from NegotiationConfigNeuralNet import *
import os
import glob
import imageio
import Experiments as exp
import tensorflow as tf
from itertools import groupby

from multiprocessing import Pool

PRED_PATH = 'predictions/brats2015_validation/'
RESULT_PATH = 'results/brats2015_validation/'
os.makedirs(RESULT_PATH, exist_ok=True)
os.makedirs(RESULT_PATH+'csv/', exist_ok=True)
GT_LIST = sorted(glob.glob(PRED_PATH+'GT/*')) # Here the files are forted by SAMPLE but not by SLICE

AGENTS = [
 'Segan_IO_TF2_brats_on_T1',
 'Segan_IO_TF2_brats_on_T1c',
 'Segan_IO_TF2_brats_on_T2', 
 'Segan_IO_TF2_brats_on_FLAIR',
 'Transfer_Brats_Flair_to_T1',
 'Transfer_Brats_Flair_to_T1_freeze_all',
 'Transfer_Brats_Flair_to_T1c',
 'Transfer_Brats_Flair_to_T1c_freeze_all',
 'Transfer_Brats_Flair_to_T2',
 'Transfer_Brats_Flair_to_T2_freeze_all'
]



In [12]:
def run_thread(input_tuple):
    print("Running thread for slice {}: {} {}".format(input_tuple[0], input_tuple[1].shape, input_tuple[2].shape))
    result = exp.run_experiment_on_list(proposals_list=[input_tuple[1]], gt_list=[input_tuple[2]], return_mean=False, agent_names=AGENTS, label_names=['Negative', 'Positive'], return_outputs=True, MAX_STEPS=10)
    result[0]['sample_id'] = input_tuple[0]
    print("thread {} finished".format(input_tuple[0]))
    return 1
    

In [13]:
for sample_id, gt_paths_group in groupby(GT_LIST, key=lambda x: int(os.path.basename(x).replace('.npy', '').split('_')[0])):
    gt_slice_paths = sorted(list(gt_paths_group), key=lambda x: int(os.path.basename(x).replace('.npy', '').split('_')[-1]))

    prediction_slices = list()
    gt_slices = list()

    for gt_sp in gt_slice_paths:
        prediction = list()
        for agent in AGENTS:
            pred_binary = np.load(gt_sp.replace('/GT/', '/{}/'.format(agent))) # Binary representation
            pred_softmax = np.concatenate([1-pred_binary, pred_binary], axis=-1) # One_hot (Not tumor, Tumor) representation
            prediction.append(pred_softmax)
        prediction = np.stack(prediction)
        prediction_slices.append(prediction)
        gt_binary = np.load(gt_sp)
        gt_slices.append(np.concatenate([1-gt_binary, gt_binary], axis=-1))
    
    thread_input = list(zip([i for i in range(len(prediction_slices))], prediction_slices, gt_slices))
    pool = Pool(processes=16)
    thread_results = pool.map(run_thread, thread_input)
    
#     results, outputs = exp.run_experiment_on_list(proposals_list=prediction_slices[:5], gt_list=gt_slices[:5], return_mean=False, agent_names=AGENTS, label_names=['Negative', 'Positive'], return_outputs=True, MAX_STEPS=10)
#     results['gt_id'] = sample_id
    
#     results.to_csv(RESULT_PATH+'csv/'+ str(sample_id) + '.csv')
#     for output in outputs:
#         for method_name, outp in output.items():
#             os.makedirs(RESULT_PATH+method_name+'/', exist_ok=True)
#             if isinstance(outp, dict):
#                 for agr_prop in outp.keys():
#                     np.save(RESULT_PATH+method_name+'/' + str(sample_id) + '_' + agr_prop + '.npy', outp[agr_prop])
#             else:
#                 np.save(RESULT_PATH+method_name+'/' + str(sample_id)+'.npy', outp.astype(np.float32))
        
    print("Done sample {}".format(str(sample_id)))
    break

Running thread for slice 0: (10, 160, 160, 2) (160, 160, 2)
Running thread for slice 2: (10, 160, 160, 2) (160, 160, 2)
Running thread for slice 4: (10, 160, 160, 2) (160, 160, 2)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Running thread for slice 6: (10, 160, 160, 2) (160, 160, 2)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Running thread for slice 8: (10, 160, 160, 2) (160, 160, 2)
Running thread for slice 10: (10, 160, 160, 2) (160, 160, 2)


...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


thread 124 finished
Running thread for slice 125: (10, 160, 160, 2) (160, 160, 2)
Proposals has full consensus, skipping...
thread 123 finished
Proposals has full consensus, skipping...
Proposals has full consensus, skipping...
thread 126 finished
thread 125 finished
Running thread for slice 127: (10, 160, 160, 2) (160, 160, 2)
Proposals has full consensus, skipping...
thread 127 finished


Process ForkPoolWorker-256:
Process ForkPoolWorker-261:
Process ForkPoolWorker-263:
Process ForkPoolWorker-255:
Process ForkPoolWorker-251:
Process ForkPoolWorker-250:
Process ForkPoolWorker-253:
Traceback (most recent call last):
Process ForkPoolWorker-260:
Process ForkPoolWorker-257:
Process ForkPoolWorker-252:
Process ForkPoolWorker-254:
Traceback (most recent call last):
Process ForkPoolWorker-264:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-258:
Process ForkPoolWorker-259:
Process ForkPoolWorker-262:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-265:
Traceback (most recent call last):
Traceback (m

KeyboardInterrupt: 

  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self.

In [15]:
os.cpu_count()

16

In [11]:
thread_input[0][0]

0